## 示例1：使用Arcpy进行GIS人口空间分布数据探索
> 本示例简单演示了通过使用arcpy的几种列出数据的方法查看中国人口数据shp文件的信息，通过游标查询单个shp文件的属性表，探索其中的字段，并进行总人口的计算。
> 本示例的数据文件在[第七次人口普查数据](https://github.com/renhai-lab/Urban-Spatial-Data-Analysis-Notebook/tree/4846a410da6c1f858ee64b02c14bdf610e08948a/4-%E7%A9%BA%E9%97%B4%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/4.2-%E3%80%90ArcGIS%20Python%E7%B3%BB%E5%88%97%E3%80%91/resource/%E7%AC%AC%E4%B8%83%E6%AC%A1%E4%BA%BA%E5%8F%A3%E6%99%AE%E6%9F%A5%E6%95%B0%E6%8D%AE)文件夹中。


> 本次数据为已处理好的分年龄、分性别的人口普查数据，来源于公众号"立方数据学社"。在'resource\第七次人口普查数据'文件夹的目录结构如下：
!['resource\第七次人口普查数据'文件夹的目录](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308121110971.png)


In [1]:
import arcpy
import os
arcpy.env.workspace = os.path.join(os.getcwd(), "resource/第七次人口普查数据")

# 此目录为多层结构，需要递归遍历
walk = arcpy.da.Walk(arcpy.env.workspace, datatype="FeatureClass")
for dirpath, dirnames, filenames in walk:
    for file in filenames:
        # print(os.path.join(dirpath, file))
        fc = os.path.join(dirpath, file)
        desc = arcpy.da.Describe(fc)
        print(desc["baseName"], desc["shapeType"])

分年龄、性别的人口_区县等级 Polygon
分年龄、性别的人口_地级市等级 Polygon
分年龄、性别的人口_省份等级 Polygon


In [2]:
# 先创建一个文件数据库
outgdb = "census.gdb"
arcpy.CreateFileGDB_management(arcpy.env.workspace, outgdb)

<Result 'C:\\Users\\hncdj\\Documents\\Python_\\Python辅助城市研究\\Urban-Spatial-Data-Analysis-For-Beginners\\4-空间数据分析\\4.2-arcpy\\resource/第七次人口普查数据\\census.gdb'>

![](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308121108933.png)

In [3]:
# 之后将所有的多边形类型的数据集都复制到这个文件数据库中
walk = arcpy.da.Walk(arcpy.env.workspace, datatype="FeatureClass")
for dirpath, dirnames, filenames in walk:
    for file in filenames:
        fc = os.path.join(dirpath, file)
        desc = arcpy.da.Describe(fc)
        if desc["shapeType"] == "Polygon":
            outfc = os.path.join(outgdb, desc["baseName"].replace("、", "_")) # 不能有特殊字符："、"
            arcpy.CopyFeatures_management(fc, outfc)

我们已经将shp文件导入到了数据库中：
![](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308121125236.png)

ArcGIS Pro中也提供了一种只能得导入方式叫[批量导入数据 (智能)](https://pro.arcgis.com/zh-cn/pro-app/latest/tool-reference/intelligence/batch-import-data.htm)，可以将 KML、KMZ、shapefile、Excel 工作表、表格文本文件、GeoJSON等文件导入地理数据库，支持文件夹和子文件夹导入，支持过滤。

In [4]:
# 查看刚刚导入的要素的相关信息
arcpy.env.workspace = os.path.join(os.getcwd(), "resource/第七次人口普查数据", "census.gdb")
fc = arcpy.ListFeatureClasses()
print("共有{}个shp文件".format(len(fc)) )

共有3个shp文件


In [5]:
print(f"第一个要素文件的名字是: {fc[0]}")

第一个要素文件的名字是: 分年龄_性别的人口_区县等级


In [6]:
# 查看第一个要素的空间参考
sr = arcpy.Describe(fc[0]).spatialReference.name
sr

'GCS_WGS_1984'

本次只探索空间数据，所以不需要进行投影。

In [7]:
# 查看第一个要素文件的字段
fields = arcpy.ListFields(fc[0])
col = [field.name for field in fields]
col

['OBJECTID',
 'Shape',
 '地名',
 '区划码',
 '县级',
 '县级码',
 '县级类',
 '地级',
 '地级码',
 '地级类',
 '省级',
 '省级码',
 '省级类',
 'F0_男',
 'F0_女',
 'F1_4_男',
 'F1_4_女',
 'F5_9_男',
 'F5_9_女',
 'F10_14_男',
 'F10_14_女',
 'F15_19_男',
 'F15_19_女',
 'F20_24_男',
 'F20_24_女',
 'F25_29_男',
 'F25_29_女',
 'F30_34_男',
 'F30_34_女',
 'F35_39_男',
 'F35_39_女',
 'F40_44_男',
 'F40_44_女',
 'F45_49_男',
 'F45_49_女',
 'F50_54_男',
 'F50_54_女',
 'F55_59_男',
 'F55_59_女',
 'F60_64_男',
 'F60_64_女',
 'F65_69_男',
 'F65_69_女',
 'F70_74_男',
 'F70_74_女',
 'F75_79_男',
 'F75_79_女',
 'F80_84_男',
 'F80_84_女',
 'F85及以上男',
 'F85及以上女',
 'Shape_Length',
 'Shape_Area']

### 下面我们通过游标查询一下分省份的人口数据

In [8]:
# 重复一下上面的步骤，过滤出分省份的要素
fc = arcpy.ListFeatureClasses("*省份*")
fc

['分年龄_性别的人口_省份等级']

In [9]:
# 查看其字段
fields = arcpy.ListFields(fc[0])
# 我们储存一下字段名
field_names = [field.name for field in fields]
field_names

['OBJECTID',
 'Shape',
 '省',
 '省级码',
 '省类型',
 'F0_男',
 'F0_女',
 'F1_4_男',
 'F1_4_女',
 'F5_9_男',
 'F5_9_女',
 'F10_14_男',
 'F10_14_女',
 'F15_19_男',
 'F15_19_女',
 'F20_24_男',
 'F20_24_女',
 'F25_29_男',
 'F25_29_女',
 'F30_34_男',
 'F30_34_女',
 'F35_39_男',
 'F35_39_女',
 'F40_44_男',
 'F40_44_女',
 'F45_49_男',
 'F45_49_女',
 'F50_54_男',
 'F50_54_女',
 'F55_59_男',
 'F55_59_女',
 'F60_64_男',
 'F60_64_女',
 'F65_69_男',
 'F65_69_女',
 'F70_74_男',
 'F70_74_女',
 'F75_79_男',
 'F75_79_女',
 'F80_84_男',
 'F80_84_女',
 'F85及以上男',
 'F85及以上女',
 'Shape_Length',
 'Shape_Area']

我们通过查询游标来查看一下是不是34个省份的数据都在

In [10]:
num = 0
with arcpy.da.SearchCursor(fc[0], "省") as cursor:
    for row in cursor:
        print(row[0])
        num += 1
print(f"一共有{num}个省份的数据")

北京市
天津市
河北省
山西省
内蒙古自治区
辽宁省
吉林省
黑龙江省
上海市
浙江省
安徽省
福建省
江西省
山东省
河南省
湖北省
湖南省
广东省
广西壮族自治区
海南省
重庆市
四川省
贵州省
云南省
西藏自治区
陕西省
甘肃省
青海省
宁夏回族自治区
新疆维吾尔自治区
江苏省
台湾省
香港特别行政区
澳门特别行政区
一共有34个省份的数据


我们再通过总人口数来验证一下数据是否正确 

In [11]:
# 首先要素表中并未统计总人口数，我们需要自己添加一个字段
arcpy.AddField_management(fc[0], "total", "LONG")

<Result 'C:\\Users\\hncdj\\Documents\\Python_\\Python辅助城市研究\\Urban-Spatial-Data-Analysis-For-Beginners\\4-空间数据分析\\4.2-arcpy\\resource/第七次人口普查数据\\census.gdb\\分年龄_性别的人口_省份等级'>

In [12]:
# 重新获取一下字段 查看是否添加成功
fields = arcpy.ListFields(fc[0])
field_names = [field.name for field in fields]
field_names

['OBJECTID',
 'Shape',
 '省',
 '省级码',
 '省类型',
 'F0_男',
 'F0_女',
 'F1_4_男',
 'F1_4_女',
 'F5_9_男',
 'F5_9_女',
 'F10_14_男',
 'F10_14_女',
 'F15_19_男',
 'F15_19_女',
 'F20_24_男',
 'F20_24_女',
 'F25_29_男',
 'F25_29_女',
 'F30_34_男',
 'F30_34_女',
 'F35_39_男',
 'F35_39_女',
 'F40_44_男',
 'F40_44_女',
 'F45_49_男',
 'F45_49_女',
 'F50_54_男',
 'F50_54_女',
 'F55_59_男',
 'F55_59_女',
 'F60_64_男',
 'F60_64_女',
 'F65_69_男',
 'F65_69_女',
 'F70_74_男',
 'F70_74_女',
 'F75_79_男',
 'F75_79_女',
 'F80_84_男',
 'F80_84_女',
 'F85及以上男',
 'F85及以上女',
 'Shape_Length',
 'Shape_Area',
 'total']

In [13]:
field_names[5:-3]

['F0_男',
 'F0_女',
 'F1_4_男',
 'F1_4_女',
 'F5_9_男',
 'F5_9_女',
 'F10_14_男',
 'F10_14_女',
 'F15_19_男',
 'F15_19_女',
 'F20_24_男',
 'F20_24_女',
 'F25_29_男',
 'F25_29_女',
 'F30_34_男',
 'F30_34_女',
 'F35_39_男',
 'F35_39_女',
 'F40_44_男',
 'F40_44_女',
 'F45_49_男',
 'F45_49_女',
 'F50_54_男',
 'F50_54_女',
 'F55_59_男',
 'F55_59_女',
 'F60_64_男',
 'F60_64_女',
 'F65_69_男',
 'F65_69_女',
 'F70_74_男',
 'F70_74_女',
 'F75_79_男',
 'F75_79_女',
 'F80_84_男',
 'F80_84_女',
 'F85及以上男',
 'F85及以上女']

In [14]:
field_names[-1]

'total'

In [15]:
# 我们需要传入分年龄的人口字段
sum_fields = field_names[5:-3] + field_names[-1:]
sum_fields

['F0_男',
 'F0_女',
 'F1_4_男',
 'F1_4_女',
 'F5_9_男',
 'F5_9_女',
 'F10_14_男',
 'F10_14_女',
 'F15_19_男',
 'F15_19_女',
 'F20_24_男',
 'F20_24_女',
 'F25_29_男',
 'F25_29_女',
 'F30_34_男',
 'F30_34_女',
 'F35_39_男',
 'F35_39_女',
 'F40_44_男',
 'F40_44_女',
 'F45_49_男',
 'F45_49_女',
 'F50_54_男',
 'F50_54_女',
 'F55_59_男',
 'F55_59_女',
 'F60_64_男',
 'F60_64_女',
 'F65_69_男',
 'F65_69_女',
 'F70_74_男',
 'F70_74_女',
 'F75_79_男',
 'F75_79_女',
 'F80_84_男',
 'F80_84_女',
 'F85及以上男',
 'F85及以上女',
 'total']

In [16]:
# 其次，通过更新游标计算总人口数
total_census = 0
with arcpy.da.UpdateCursor(fc[0], sum_fields) as cursor:
    for row in cursor:
        row[-1] = sum(row[:-1])
        total_census += row[-1]
        # cursor.updateRow(row) # 先注释掉，测试代码成功后再运行
print(total_census)

1409778724


可以看到14亿人口的数据是正确的，我们可以将其写入到一个"total"的字段中了。


In [17]:
# 其次，通过更新游标计算总人口数
total_census = 0
with arcpy.da.UpdateCursor(fc[0], sum_fields) as cursor:
    for row in cursor:
        row[-1] = sum(row[:-1])
        total_census += row[-1]
        cursor.updateRow(row)

![](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/202308121201974.png)

还可以继续探索的其他示例：
1. 单独筛选出指定省类型的省份，比如只查看直辖市的人口。
2. 深入探索人口数据：人口的年龄结构、空间分布等，制作人口年龄结构图。链接到~~python示例里~~。
3. 继续探索分区县、分地级市的人口数据。
结合后续教程：
1. 结合mp制图模块和符号系统批量出空间分布图。